In [1]:
#-*- coding:utf-8 -*-

#import python packages
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_samples
from sklearn.cluster import KMeans

from sklearn.utils.testing import ignore_warnings
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import *
from sklearn.cluster import *

from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from operator import itemgetter
from operator import attrgetter

from pyjarowinkler import distance
from collections import Counter

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import nltk

import math
import time

import csv
import sys

import re
import io
import os

start_time = time.time()

#전처리 함수 정의부
def remove_string_special_characters(s):
    
    stripped = re.sub('[^a-zA-z\s]', '', s)
    
    stripped = re.sub('_', '', stripped)
    
    stripped = re.sub('\s+', ' ', stripped)
    
    stripped = stripped.strip()
    
    if stripped != '':
        return stripped.lower()
    
#클래스 정렬 함수 정의부
def multisort(xs, specs):
    
    for key, reverse in reversed(specs):
        
        xs.sort(key=attrgetter(key), reverse=reverse)
        
    return xs

#속성집합 추출 함수 정의부
#키워드 매개변수(입력csv path, 속선집합 포함 출력csv path, 추출할 단어 수)
def extractive_keyword(path,database_update_path,extract_word_num=20):
    
    reviewee = pd.read_csv(path, encoding='latin1')
    count,temp = len(reviewee),[]

    for i in range(count):
        
        temp_intro = reviewee['submitter_intro'][i]

        textrank_text = ''

        for c in (keywords(temp_intro, words=extract_word_num, lemmatize=True).split('\n')):
            
            textrank_text += (c+ " ")

        temp.append(textrank_text)

    reviewee['submitter_attribute']=temp
    
    #return type : pandas.dataframe
    return reviewee

reviewee=extractive_keyword(path='../reviewee/submitter_10.csv',
                                database_update_path='../../reviewee/sybmitter_update_100.csv',
                                extract_word_num=20)
reviewee[:3]

,date,submitter_orcid,submitter_name,submitter_institution,submitter_email,submitter_title_doi,submitter_title,submitter_intro,submitter_author1_name,submitter_author1_institution,...,submitter_author8_name,submitter_author8_institution,submitter_author8_email,submitter_author9_name,submitter_author9_institution,submitter_author9_email,submitter_author10_name,submitter_author10_institution,submitter_author10_email,submitter_attribute
0,20200101,326,Lu Qin,"The Chinese University of Hong Kong, Hong Kong...",lqin@se.cuhk.edu.hk,NaN,Diversifying Top-K Results,ABSTRACT This paper proposes a general framewo...,Lu Qin,"The Chinese University of Hong Kong, Hong Kong...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,distance similarities match sequences subseque...
1,20200102,897,Haohan Zhu,Department of Computer Science Boston University,zhu@cs.bu.edu,NaN,A Generic Framework for Efficient and Effectiv...,ABSTRACT Top-k query processing finds a list o...,Haohan Zhu,Department of Computer Science Boston University,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,result algorithms searches problems solution b...
2,20200103,1387,Stefan Aulbach,"Technische Universit?t M?nchen, Germany",stefan.aulbach@in.tum.de,NaN,A Comparison of Flexible Schemas for Software ...,ABSTRACT A multi-tenant database system for So...,Stefan Aulbach,"Technische Universit?t M?nchen, Germany",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,table column mappings tenants performing queri...


In [ ]:
#-*- coding:utf-8 -*-
#전문성 검사 함수 정의부
#키워드 매개변수(입력csv path, 투고원고 DataFrame, i번째 투고 원고, 추천할 심사자 수, 실루엣값 계산 범위 지정)
def professionalism(path,extractive_keyword_result,reviewee_index,top_limit,silhouette_range=25):
    
    #인자 전달
    reviewee,index,top=extractive_keyword_result,reviewee_index,top_limit
    temp_id,temp_doi = 0,''
    
    #투고 원고 열 정보 추출
    temp_title = reviewee.loc[index]['submitter_title']
    temp_attribure = reviewee.loc[index]['submitter_attribute']
    
    # 심사자풀 Read
    reviewer_attr = pd.read_csv(path, encoding='latin1')
    
    # 투고원고 dataframe 맨 윗줄 추가
    reviewer_attr.loc[-1] = [str(temp_id),temp_doi,temp_title,temp_attribure]
    reviewer_attr.index += 1
    reviewer_attr.sort_index(inplace=True)
    reviewer = reviewer_attr['reviewer_paper_attribure']
    
    jac_token,jac,cos,avg=[],[],[],[]

    # jaccard_distance
    for t in range(len(reviewer)):        
        jac_token.append(set(nltk.ngrams((nltk.word_tokenize(reviewer[t])), n=1)))    
    for j in range(len(reviewer)):
        jac.append(1-(nltk.jaccard_distance(jac_token[0], jac_token[j])))

    # cosine_similarity
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(reviewer)
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=[i for i in reviewer])
    cos=cosine_similarity(df, df)[0].tolist()

    for i in range(len(jac)):
        avg.append((jac[i] + cos[i])/2)
    reviewer_attr['sim']=avg
    
    reviewer_attr2 = reviewer_attr.sort_values(by=['sim'], axis=0, ascending=False)
    reviewer_attr3=reviewer_attr2[~reviewer_attr2.duplicated(['reviewer_orcid'],keep=False) == True]
    reviewer_attr4 = reviewer_attr2[reviewer_attr2.duplicated(['reviewer_orcid'],keep='last') == True]
    reviewer_attr5=pd.concat([reviewer_attr3,reviewer_attr4])
    reviewer_attr5.rename(columns = {'reviewer_paper_attribure' : 'reviewer_paper_feature'}, inplace = True)
    reviewer2 = list(reviewer_attr5['reviewer_paper_feature'])
    
    # 속성집합 20개 제한
    reviewer_attribute=[]
    for i in range(len(reviewer)):
        a=((reviewer[i]).split(' '))
        b=a[:20]
        temp=[]
        for j in range(20):
            temp.append(b[j])
            temp += (str(b[j]) + ',')
        reviewer_attribute.append(temp[:-1])
    
    common_texts_and_tags = [
        (text, [f"str_{i}",]) for i, text in enumerate(reviewer_attribute)
    ]
    
    TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in common_texts_and_tags]
    
    model = Doc2Vec(TRAIN_documents)
    
    for text, tags in common_texts_and_tags:
        trained_doc_vec = model.docvecs[tags[0]]
        inferred_doc_vec = model.infer_vector(text)

    dtm_df=[]
    for text, tags in common_texts_and_tags:

        inferred_doc_vec = model.infer_vector(text)

        dtm_df_temp=[]
        for text2, tag2s in common_texts_and_tags:

            inferred_doc_vec = model.infer_vector(text2)

            sim = word_vectors.wmdistance(text, text2)

            dtm_df_temp.append(sim)
        dtm_df.append(dtm_df_temp)

    dissim_df = pd.DataFrame(data=dtm_df)
    dissimilarity=dissim_df.values

    model1 = AgglomerativeClustering(n_clusters=8, affinity='euclidean', linkage='ward')
    model1.fit(dissimilarity)
    labels1 = model1.labels_
    dissim_df['cluster']=labels1
    
    cluster_reviewer = reviewer_attr5[reviewer_attr5['cluster'] == reviewer_attr5.loc[0]['cluster']]

    cluster_reviewer2 = cluster_reviewer.sort_values(by=['sim'], axis=0, ascending=False)
    
    professionalism=cluster_reviewer2.iloc[0:top+1] #1
    
    #return type : pandas.dataframe
    return dissim_df
    
#전문성검사
#키워드 매개변수(입력csv path, 투고원고 DataFrame, i번째 투고 원고, 추천할 심사자 수, 실루엣값 계산 범위 지정)
i=0
print(str(i)+'번째')
aaa = professionalism(path='./reviewer_attribute_937.csv',
                     extractive_keyword_result=reviewee,
                     reviewee_index=i,
                     top_limit=10,
                     silhouette_range=25)
#return type : pandas.dataframe

# reviewer


In [ ]:
def interest(co_author_path, reviewer_information_path, co_author_network_path, professionalism_result, extractive_keyword_result, reviewee_index,matrix_multifly_count):
    
    #매개변수 전달
    path1,path2=co_author_path,reviewer_information_path
    network_path=co_author_network_path
    temp,reviewee = professionalism_result,extractive_keyword_result
    index,multifly=reviewee_index,matrix_multifly_count

    #import Dataset
    co_author_csv = pd.read_csv(path1, encoding='latin1')
    
    #merge coauthor & 전문성 검사 결과
    co_author_df = co_author_csv.merge(temp, on=['reviewer_orcid'])  
    co_author_df2 = co_author_df.iloc[:]['reviewer_name'].tolist()
    
    #import network
    try :
        #read network csv
        network_csv = pd.read_csv(network_path, encoding='latin1',index_col=0)
    except FileNotFoundError :
        df1=pd.read_csv('./network/network0704.csv')
        df2=pd.read_csv('./network/network0704_2.csv')

        tmp=[1]
        tmp.extend(i*2 for i in range(1,11))

        for k in range(1,len(df1.columns)):
            a=df1.columns[k]
            a1=df2.loc[df2['reviewer_coauthor_title']==a]
            a_list=[]
            for i in range(len(tmp)):
                a_list.append(a1.iloc[0][tmp[i]])
            for i in range(len(df1)):
                if (df1.iloc[i,0] in a_list) is True :df1.iloc[i,k]=1
                else :df1.iloc[i,k]=0

        mat=(df1.values)[:,1:]
        mat2=mat.T
        mat3=np.dot(mat,mat2)
        for i in range(len(mat3)):
            mat3[i,i]=0
        fof=np.dot(mat3,mat3)
        
        df1_index=(df1.iloc[:,0]).tolist()
        df1_col=(df1.columns)[1:]
        network_csv=pd.DataFrame(data=mat3,index=df1_index,columns=df1_index)
        network_csv.to_csv(network_path)
    
    display(network_csv)
    reviewer_list=(network_csv.index).tolist()
    
    reviewee_list=[]
    reviewee.fillna(0, inplace=True)
    for i in range(1,11):
        col_index = (i*3)+5
        if reviewee.loc[index][col_index] != 0:
            reviewee_list.append(reviewee.loc[index][col_index])
    
    co_rel_df = pd.DataFrame(
        columns=[i for i in reviewer_list],
        index=[j for j in reviewee_list])
    
    for i in range(len(reviewee_list)):
        indet_temp=reviewer_list.index(reviewee_list[i])
        co_rel_df.iloc[i,indet_temp]=1
    co_rel_df.fillna(0, inplace=True)
    
    display(co_rel_df.iloc[:,19:])
    
    network_csv_v=network_csv.values
    for i in range(multifly):
        network_csv_v = network_csv_v.dot(network_csv_v)

    co_rel_df_v=co_rel_df.values
    fof = co_rel_df_v.dot(network_csv_v)
    
    df_fof=pd.DataFrame(data=fof,
                 index=(co_rel_df.index).tolist(),
                 columns=(network_csv.index).tolist())
    
    display(df_fof.iloc[:,[20,21,22,23,24,26,27,28,29,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]])
    df_fof_series =df_fof.loc[:, (df_fof != 0).any(axis=0)]
    df_fof_series_list=(df_fof_series.columns).tolist()
    
    reviewer_list1 = list(set(co_author_df2).difference(df_fof_series_list))
    
    co_inst_csv = pd.read_csv(path2, encoding='latin1')

    co_inst_df = co_inst_csv.merge(temp, on=['reviewer_orcid'])

    reviewee_list2=[]
    for i in range(1,11):
        col_index = (i*3)+6
        if reviewee.loc[index][col_index] != 0:
            reviewee_list2.append(reviewee.loc[index][col_index])
    
    reviewer_list2,reviewer_inst_list=[],[]
    for j in range(len(co_inst_df)):#
        inst_list_temp=[]
        reviewer_list2.append(co_inst_df['reviewer_name'][j])#
        reviewer_inst_list.append(co_inst_df['reviewer_institution'][j])#

    inst_rel_df = pd.DataFrame(
        columns=[i for i in reviewer_list2],#
        index=[j for j in reviewee_list])#

    for i in range(len(reviewee_list2)):
        for j in range(len(reviewer_inst_list)):
            if (reviewee_list2[i] == reviewer_inst_list[j]) : inst_rel_df.iloc[i, j] = 1
            else : inst_rel_df.iloc[i, j] = 0
                
    #-----------------
    
#     display(inst_rel_df.iloc[:,[750,751,752,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,1,2,3,4,5,6,7,8,9,10]])
    
    #-----------------
    
    inst_rel_df_series = inst_rel_df.loc[:, (inst_rel_df != 0).any(axis=0)]
    inst_rel_df_series_list=(inst_rel_df_series.columns).tolist()
    
    reviewer_list2 = list(set(reviewer_list2).difference(inst_rel_df_series_list))
    
    reviewer_rank_list = list(set(reviewer_list1).intersection(reviewer_list2))

    id_index,sim_index,count_index,title_index=[],[],[],[]
    reviewer_rank = pd.DataFrame({'reviewer_name': reviewer_rank_list})
    
    for i in range(len(reviewer_rank)):
        for j in range(len(co_author_df)):
            if reviewer_rank.loc[i]['reviewer_name'] == co_author_df.loc[j]['reviewer_name'] :
                id_index.append(int(co_author_df.iloc[j]['reviewer_orcid']))
                sim_index.append(co_author_df.iloc[j]['sim'])
                title_index.append(co_author_df.iloc[j]['reviewer_title'])
                break
            if reviewer_rank.loc[i]['reviewer_name'] == co_inst_df.loc[j]['reviewer_name'] :
                count_index.append(co_inst_df.iloc[j]['count'])
    
    aaa=[]
    for i in range(len(reviewer_rank)):
        aaa.append(0)
    
    reviewer_rank['reviewer_orcid']=id_index
    reviewer_rank['title']=title_index
    reviewer_rank['sim']=sim_index
    reviewer_rank['count']=aaa
    
    reviewer_rank2 = reviewer_rank.sort_values(by=['sim'], axis=0, ascending=False)
                
    #return type : pandas.dataframe
    return reviewer_rank2

In [ ]:
def save_csv(output_path,extractive_keyword_result,professionalism_result,reviewee_index,top_limit):
    
    path,ee_num,top=output_path,reviewee_index,top_limit
    reviewee,reviewer_rank_name=extractive_keyword_result,professionalism_result
    
    export_data=[]
    for i in range((top*2)):
        #reviewee
        temp=[]
        temp.append(reviewee.iloc[(1//top*2)+ee_num]['submitter_title'])
        temp.append(reviewee.iloc[(1//top*2)+ee_num]['date'])
        temp.append(reviewee.iloc[(1//top*2)+ee_num]['submitter_name'])
        #reviewer_rank_name
        temp.append(reviewer_rank_name.iloc[i]['reviewer_name'])
        temp.append(reviewer_rank_name.iloc[i]['reviewer_orcid'])
        temp.append(reviewer_rank_name.iloc[i]['title'])
        temp.append(reviewer_rank_name.iloc[i]['sim'])
        temp.append(reviewer_rank_name.iloc[i]['count'])
        #export_data
        export_data.append(temp)
        
    try :
        export_csv = pd.read_csv(path,index_col=0)
    except FileNotFoundError :
        export_csv = pd.DataFrame([],columns=['submitter_title','date','submitter_name','reviewer_name','reviewer_orcid','title','sim','count'])
    
    for i in range(len(export_data)):
        export_csv.loc[len(export_csv)] = export_data[i]
    
    export_csv2 = export_csv.sort_values(by=['sim'], axis=0, ascending=False)
    
    export_csv2.to_csv(path)

In [ ]:
def equl_distribution(input_csv_path, output_csv_path):
    
    #read csv
    export_csv2 = pd.read_csv(input_csv_path,index_col=0)
    
    class Paper:
        
        def __init__(self, title, date, submitter, reviwer_name, reviwer_orcid, title2, sim, count):
            self.title = title
            self.date = date
            self.submitter = submitter
            self.reviwer_name = reviwer_name
            self.reviwer_orcid = reviwer_orcid
            self.reviwer_title = title2
            self.sim = sim
            self.count = count

        def __repr__(self):
            return repr((self.title, self.date, self.submitter, self.reviwer_name, self.reviwer_orcid, self.reviwer_title, self.sim, self.count))

    papers,objs=[export_csv2.iloc[i].tolist() for i in range(len(export_csv2))],[]

    for paper in papers:
        objs.append(Paper(*paper))
    
    o = (multisort(list(objs), (('date', False), ('sim', True))))
    
    final_list=[]
    for i in range(0,len(export_csv2),6) :
        temp_list=[]
        for t in range(6):
            if len(temp_list) == 3:break
            else :
                temp = i + t
                if (o[temp].count) < 3 :
                    o[temp].count += 1
                    for j in range(0+temp, len(export_csv2)) :
                        if (o[temp].reviwer_name == o[j].reviwer_name) :
                            o[j].count += 1
                    o[temp].count -= 1
                    class_1=(str(o[temp]))[1:-1]
                    class_2=class_1.replace('\'','')
                    class_3=class_2.split(', ')
                    temp_list.append(class_3)
        final_list.extend(temp_list)
        
    final=pd.DataFrame(final_list,columns=['submitter_title','date','submitter_name','reviewer_name','reviewer_orcid','title','sim','count'])
    final.to_csv(output_csv_path)

In [ ]:
reviewee=extractive_keyword(path='../reviewee/submitter_10_k.csv',
                            database_update_path='../reviewee/update/sybmitter_update_100.csv',
                            extract_word_num=30)
#return type : pandas.dataframe
print('[extractive_keyword | {time:2.5f}]'.format(time=time.time() - start_time))
display(reviewee)
print('---------------1---------------')

In [ ]:
reviewer=professionalism(path='../reviewer_pool/reviewer_attribute_937.csv',
                         extractive_keyword_result=reviewee,
                         reviewee_index=0,
                         top_limit=20,
                         silhouette_range=25)
#return type : pandas.dataframe
display(reviewer.iloc[:,[0,2,3,4,5]])
print('--------------2----------------')

In [ ]:
reviewer_rank = interest(
    co_author_path='../reviewer_pool/reviewer_coauthor_937.csv',
    reviewer_information_path='../reviewer_pool/reviewer_information_937.csv',
    co_author_network_path='../co_author_network/network.csv',
    professionalism_result=reviewer,
    extractive_keyword_result=reviewee,
    reviewee_index=0,
    matrix_multifly_count=1)
#return type : pandas.dataframe
display(reviewer_rank)
# reviewer_rank.to_csv('./a.csv')
print('--------------3----------------')


In [ ]:
save_csv(output_path='../algorithm_output/export_csv_937.csv',
         extractive_keyword_result=reviewee,
         professionalism_result=reviewer_rank,
         reviewee_index=0,
         top_limit=3)
display(save_csv)


print('-------------------11------------------')

In [ ]:
equl_distribution(input_csv_path='../algorithm_output/export_csv_937.csv',
             output_csv_path='../algorithm_output/final_csv_937.csv')

print('[equl_distribution | {time:2.5f}]'.format(time=time.time() - start_time))